### Import od necessary libraries

In [1]:
import pyodbc as odbc
import pandas as pd
import sqlalchemy as sa

### Creating connection with SQL Server Database using pyodbbc + SQLAlchemy

In [2]:
DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = 'DESKTOP-6DMIBIJ'
DATABASE_NAME = 'Olist'

In [3]:
connection_string = f"mssql+pyodbc://@{SERVER_NAME}/{DATABASE_NAME}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"


In [4]:
engine = sa.create_engine(connection_string)

### First insight in Data 

In [147]:
df_customers = pd.read_sql('SELECT* FROM dbo.customers', engine)
df_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,6273,osasco,SP
1,000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,35550,itapecerica,MG
2,0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,29830,None,None
3,0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,39664,mendonca,MG
4,000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,4841,None,None


In [87]:
df_geolocation = pd.read_sql('SELECT * FROM dbo.geolocation', engine)
df_geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [45]:
print(f"Number of rows in Customers table: {len(df_customers)}")
print(f"Number of rows in Geolocation table: {len(df_geolocation)}")

Number of rows in Customers table: 99441
Number of rows in Geolocation table: 1000163


In [46]:
df_customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [9]:
df_geolocation.dtypes

geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object

In [47]:
df_customers.isna().sum()

customer_id                     0
customer_unique_id              0
customer_zip_code_prefix        0
customer_city               31474
customer_state              31474
dtype: int64

In [30]:
df_geolocation["geolocation_city"].value_counts()

geolocation_city
sao paulo                    160718
rio de janeiro                62151
belo horizonte                27805
curitiba                      16593
porto alegre                  13521
                              ...  
vitorinos - alto rio doce         1
sao sebastiao dos torres          1
sao sebastiao da vitória          1
mantiqueira do palmital           1
itapucá                           1
Name: count, Length: 7560, dtype: int64

In [11]:
df_geolocation.isna().sum()

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

Trzeba zrobić transformacje na danych - żeby usunąć wszystkie hiszpańskie znaki, następnie sprawdze ile jest takich przypadków że jest inne miasto i stan dla tego samego zip code

In [88]:
replace_dict = {
    'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'ñ': 'n', 'ç': 'c',
    'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', 'Ñ': 'N', 'Ç': 'C'
}

for old_val, new_val in replace_dict.items():
    df_geolocation["geolocation_city"] = df_geolocation["geolocation_city"].str.replace(old_val, new_val)

In [89]:
df_geolocation["geolocation_city"] = df_geolocation["geolocation_city"].str.replace(r"[-'']", '', regex=True)
# df_geolocation["geolocation_state"] = df_geolocation["geolocation_state"].str.replace(r"[áéíóúñÁÉÍÓÚÑÇ-']", '', regex=True)

In [50]:
df_geolocation.loc[df_geolocation["geolocation_zip_code_prefix"] == 88052] #-> Sprawdzenie czy usunęło hiszpańskie znaki

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
812257,88052,-27.453479,-48.451761,florianopolis,SC
812310,88052,-27.503649,-48.491333,florianopolis,SC
822090,88052,-27.460366,-48.449849,florianopolis,SC
822108,88052,-27.506671,-48.492032,florianopolis,SC
822164,88052,-27.453479,-48.451761,florianopolis,SC
822216,88052,-27.506777,-48.464764,florianopolis,SC
822225,88052,-27.481914,-48.463108,florianopolis,SC
822318,88052,-27.496083,-48.460492,florianopolis,SC
822480,88052,-27.460937,-48.451187,florianopolis,SC
822494,88052,-27.498236,-48.459090,florianopolis,SC


1.  Z 'olist_customer_dataset' usunąć niektóre customer_city i customer_state, na podstawie zachowanego zip_code będzie można je uzupełnić 
odpowiadającymi wartościami z tabeli 'olist_geolocation_dataset'


Najpierw sprawdzamy ile customer_zip_code_prefix jest takich które mają WSZYSTKIE city i state jako NULL</br>
Kod SQL który wyświetla te zip_cody i ile razy występują</br>
  SELECT  c.customer_zip_code_prefix, COUNT(c.customer_zip_code_prefix)  FROM customers c
  WHERE NOT EXISTS (SELECT 1 FROM dbo.customers c2 WHERE c.customer_zip_code_prefix = c2.customer_zip_code_prefix AND c2.customer_city IS NOT NULL)
  GROUP BY c.customer_zip_code_prefix
  ORDER BY COUNT(c.customer_zip_code_prefix) DESC

In [148]:
zip_code_not_empty = df_customers[(df_customers['customer_state'].notna()) & (df_customers['customer_city'].notna())]['customer_zip_code_prefix'].unique() 
# ODPOWIADAJACY KOD SQL POWYZSZEJ LINII:
# SELECT COUNT(DISTINCT customer_zip_code_prefix) FROM dbo.customers 
# WHERE customer_state IS NOT NULL AND customer_city IS NOT NULL
len(zip_code_not_empty)
filtered = df_customers[~df_customers['customer_zip_code_prefix'].isin(zip_code_not_empty)]
result = filtered.groupby("customer_zip_code_prefix").size().reset_index(name="l")
print(result["l"].sum()) # -> Liczba wierszy dla customer_zip_code dla których nie ma ani jednego uzupełnionego wiersza w customers (state i country)
result.sort_values(by = ["l"], ascending = False)

1692


,customer_zip_code_prefix,l
335,18606,6
149,4657,6
1014,78115,6
172,5203,5
12,1331,5
...,...,...
1209,96460,1
1210,96540,1
1211,96824,1
1212,96843,1


Jest 1246 unikalnych customer_zip_code_prefix z tabeli df_customers które nie mają ani jednego uzupełnionego wiersza dla state i city, które razem dają 1692 wiersze

Teraz trzeba by dla tych zip codeow ktore sa powyzej zaciagnac dane z tabeli geolocation (bo dla nich dane sie nie powielają z customers)

In [149]:
zip_code_list_of_full_nulls = result["customer_zip_code_prefix"].to_list()
zip_code_list_of_full_nulls[:5]

[1003, 1016, 1021, 1022, 1023]

Teraz sprawdzenie ile jest takich przypadkow gdzie sa rozne city dla tego samego zip_code, weryfikacja czy funkcja która bierze te drugie city i state jest w ogole potrzebna

In [150]:
df_nunique_city = df_geolocation.groupby("geolocation_zip_code_prefix")["geolocation_city"].nunique()
df_nunique_city[df_nunique_city > 2]

geolocation_zip_code_prefix
9780     3
13318    3
13454    3
13855    3
17970    3
25936    3
27163    3
28950    3
28993    3
35315    3
35411    3
35413    3
36206    3
38295    3
38749    3
42820    3
42841    3
45816    3
45936    4
55485    3
57925    3
65935    4
68695    3
72220    3
72231    3
72270    3
76958    3
85139    4
87365    3
Name: geolocation_city, dtype: int64

29 jest różnych zip code'ow, dla ktorych jest rozne miasto (zazwyczaj jest i tak to samo miasto, tylko inaczej zapisane)

In [54]:
df_geolocation.loc[df_geolocation["geolocation_zip_code_prefix"] == 42850]["geolocation_city"].value_counts()

geolocation_city
dias davila     185
dias d avila     13
Name: count, dtype: int64

Teraz można przystąpić to tworzenia funkcji która zwraca dla zip code które mają kilka miast - żeby zwracało drugie miasto </br>
To narazie haszujemy i zwracamy pierwsze wystąpienie .fillna()

In [ ]:
"""
def corresponding_value(group):
    if len(group) == 1:
        return group.iloc[0]
    else:
        return group.iloc[1]
"""
# df_geo = df_geolocation[["geolocation_zip_code_prefix","geolocation_city"]]
# df_geo = df_geo.groupby(by = "geolocation_zip_code_prefix")#.apply(corresponding_value)

In [151]:
# DZIALA NA WPROWADZNIE PIEWSZEJ WARTOŚCI WYSTĄPIENIA 

print(f"Przed uzupełnieniem fragmentu brakujących danych: {(df_customers["customer_state"].notna().sum())}")
mask = df_customers["customer_zip_code_prefix"].isin(zip_code_list_of_full_nulls)
df_customers.loc[mask,"customer_state"] = df_customers.loc[mask,"customer_state"].fillna(df_geolocation["geolocation_state"])
df_customers.loc[mask,"customer_city"] = df_customers.loc[mask,"customer_city"].fillna(df_geolocation["geolocation_city"])
df_customers[mask]
print(f"Po uzupełnieniu brakujących danych: {df_customers["customer_state"].notna().sum()}")
# df_customers.loc[mask] = df_customers.loc[mask,"customer_state"].fillna(df_geo_second_choice["geolocation_state"])
# df_customers.head()

Przed uzupełnieniem fragmentu brakujących danych: 67967
Po uzupełnieniu brakujących danych: 69659


69659 - 67967 = 1692 -> Zgadza się, tyle było wartości customer_zip_code_prefix które nie miały ŻADNEGO uzupełnionego wiersza, a teraz są uzupełnione

Potem sprawdzamy ile jest takich customer_zip_code_prefix które mają wpisane NULL w city i state, ale mają też uzuepłnione dane w innym wierszu</br>
   SELECT  c.customer_zip_code_prefix, COUNT(c.customer_zip_code_prefix)  FROM customers c
  WHERE EXISTS (SELECT 1 FROM dbo.customers c2 WHERE c.customer_zip_code_prefix = c2.customer_zip_code_prefix AND c2.customer_city IS NOT NULL)
  GROUP BY c.customer_zip_code_prefix
  ORDER BY COUNT(c.customer_zip_code_prefix) DESC

In [20]:
df_customers["customer_city"].notna().sum()

np.int64(69659)

In [30]:
len(df_customers)

99441

In [21]:
print(f"Pozostałe brakujące dane dla customer_state: {df_customers["customer_state"].isna().sum()}")
print(f"Pozostałe brakujące dane dla customer_city: {df_customers["customer_city"].isna().sum()}")

Pozostałe brakujące dane dla customer_state: 29782
Pozostałe brakujące dane dla customer_city: 29782


In [152]:
# df_customers["customer_city"] = df_customers.loc[df_customers["customer_city"].isna()].groupby(by="customer_zip_code_prefix")["customer_city"].transform("first") # -> Ten obiekt trzeba uzupełnić danymi
df_customers["customer_state"] = df_customers.groupby(by="customer_zip_code_prefix")["customer_state"].transform("first")
df_customers["customer_city"] = df_customers.groupby(by="customer_zip_code_prefix")["customer_city"].transform("first")

print(f"Pozostałe brakujące dane dla customer_state:  {df_customers["customer_state"].isna().sum()}")
print(f"Pozostałe brakujące dane dla customer_city: {df_customers["customer_city"].isna().sum()}")


Pozostałe brakujące dane dla customer_state:  0
Pozostałe brakujące dane dla customer_city: 0


In [63]:
df_customers[df_customers["customer_zip_code_prefix"] == 29830]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
2,0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,29830,nova venecia,ES
7541,138e80e19b6bc2bf940a17b938a67295,1015173479527a15ccb63606a74f404e,29830,nova venecia,ES
9002,172f6c5da4c886b4013514d6de6b44b8,a34f3b7088cb15df6122dcc74cb10e72,29830,nova venecia,ES
9650,18cdebd5cfe07beb5258254ff1f8fe85,fe0617cc03bb1fdefeb9e76e7d316548,29830,nova venecia,ES
37486,60d825970926bc552c3299d2d1b5041a,e3feffb41a633b6f649b9cd7762f21d9,29830,nova venecia,ES
40480,68b7509c1f04aa6c537f935bb18b9645,6a875e0207d93e4c06f193810c6c2cd0,29830,nova venecia,ES
48453,7cf5c3cc0ca95110719e8762edf953a2,b7f5bef3510128b51df4751fa0e023da,29830,nova venecia,ES
50268,819d8a9b81c0d3e3204e5b80252660ca,1015173479527a15ccb63606a74f404e,29830,nova venecia,ES
51962,860ddc7aa11dedbcd7d870aead27ce0c,86f88192e35af3f92bd322ba84e637ce,29830,nova venecia,ES
53139,891deeb7a50f52a5a2a5970152b3864e,b171945c68776b1434cf56b06c6e41a5,29830,nova venecia,ES


In [153]:
df_customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [ ]:
#df['new_col'] = range(1, len(df) + 1)
#df.to_sql("cust_test", engine, schema = "dbo" , index = False)

302

5. Usunięcie dla niektórych wartości z kolumny seller_city z tabeli 'olist_sellers_dataset' i uzupełnienie ich na podstawie zachowanego seller_zip_code_prefix

In [90]:
df_geolocation.head(1)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP


In [91]:
df_sellers = pd.read_sql('SELECT * FROM dbo.sellers', engine)
df_sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,0015a82c2db000af6aaaf3ae2ecb0532,9080,santo andre,SP
1,001cca7ae9ae17fb1caed9dfb1094831,29156,cariacica,ES
2,001e6ad469a905060d959994f1b41e4f,24754,sao goncalo,RJ
3,002100f778ceb8431b7a1020ff7ab48f,14405,None,SP
4,003554e2dce176b5555353e4f3555ac8,74565,goiania,GO


In [94]:
df_sellers.isna().sum()

seller_id                   0
seller_zip_code_prefix      0
seller_city               705
seller_state                0
dtype: int64

In [106]:
#filtered = df_sellers[df_sellers["seller_city"].isna()]
#filtered["seller_city"]
mask = df_sellers["seller_city"].isna()
df_sellers.loc[mask]

,seller_id,seller_zip_code_prefix,seller_city,seller_state
3,002100f778ceb8431b7a1020ff7ab48f,14405,None,SP
5,004c9cd9d87a3c30c522c48c4fc07416,14940,None,SP
7,00ab3eff1b5192e5f1a63bcecfee11c8,4164,None,SP
13,011b0eaba87386a2ae96a7d32bb531d1,17580,None,SP
18,0176f73cc1195f367f7b32db1e5b3aa8,14940,None,SP
...,...,...,...,...
3069,fe8055980a4ff7f64ed889c2b5926929,81925,None,PR
3070,fe87f472055fbcf1d7e691c00b1560dc,13085,None,SP
3082,ff314fa6033cc68ec451c47aee2d6ba4,5424,None,SP
3089,ffc470761de7d0232558ba5e786e57b7,7091,None,SP


In [115]:
#df_sellers["seller_city"][~df_sellers["seller_id"].isin(filtered["seller_id"].to_list())].isna().sum()
df_sellers.head(1)

,seller_zip_code_prefix,seller_id,seller_city,seller_state
0,9080,0015a82c2db000af6aaaf3ae2ecb0532,santo andre,SP


In [113]:
df_geolocation.head(1)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP


In [118]:
# df_geolocation.set_index("geolocation_zip_code_prefix", inplace = True)
# df_sellers.set_index("seller_zip_code_prefix", inplace = True)

df_geolocation.reset_index(inplace = True)
df_sellers.reset_index(inplace = True)

In [132]:
mapping = df_geolocation.groupby("geolocation_zip_code_prefix")["geolocation_city"].first()
mapping

geolocation_zip_code_prefix
1001           sao paulo
1002           sao paulo
1003           sao paulo
1004           sao paulo
1005           sao paulo
              ...       
99960            charrua
99965         agua santa
99970            ciriaco
99980    david canabarro
99990          muliterno
Name: geolocation_city, Length: 19015, dtype: object

In [134]:
df_sellers[mask]

,seller_zip_code_prefix,seller_id,seller_city,seller_state
3,14405,002100f778ceb8431b7a1020ff7ab48f,None,SP
5,14940,004c9cd9d87a3c30c522c48c4fc07416,None,SP
7,4164,00ab3eff1b5192e5f1a63bcecfee11c8,None,SP
13,17580,011b0eaba87386a2ae96a7d32bb531d1,None,SP
18,14940,0176f73cc1195f367f7b32db1e5b3aa8,None,SP
...,...,...,...,...
3069,81925,fe8055980a4ff7f64ed889c2b5926929,None,PR
3070,13085,fe87f472055fbcf1d7e691c00b1560dc,None,SP
3082,5424,ff314fa6033cc68ec451c47aee2d6ba4,None,SP
3089,7091,ffc470761de7d0232558ba5e786e57b7,None,SP


In [157]:
df_sellers.loc[mask,"seller_city"] = df_sellers.loc[mask,"seller_city"].fillna(df_sellers["seller_zip_code_prefix"].map(mapping))
df_sellers[df_sellers["seller_zip_code_prefix"] == 76804]

,seller_zip_code_prefix,seller_id,seller_city,seller_state
606,76804,3364a91ec4d56c98e44174de954b94f6,porto velho,RO


In [144]:
df_sellers[df_sellers["seller_city"].isna()]

,seller_zip_code_prefix,seller_id,seller_city,seller_state
1088,82040,5962468f885ea01a1b6a97a218797b0a,NaN,PR


Jest 1 seller_city dla którego nie ma odpowiednika w geolocation - ten wiersz pewnie trzeba będzie usunąć

2. Usunięcie dla niektórych wartości z kolumny payment_value z tabeli 'olist_order_payments_dataset', tą wartość będzie można obliczyć 
z tabeli 'olist_order_items_dataset' sumując wartości price + freight_value dla konkretnego order_id

In [5]:
df_order_payments = pd.read_sql('SELECT * FROM dbo.order_payments', engine)
df_order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,NaN
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,NaN


In [6]:
df_order_items = pd.read_sql('SELECT * FROM dbo.order_items', engine)
df_order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [7]:
df_order_items["payment_value"] = df_order_items["price"] + df_order_items["freight_value"]

In [8]:
print(len(df_order_payments))
df_order_payments["order_id"].nunique()

103886


99440

In [9]:
for i in df_order_payments.columns:
    print(f"Liczba pustych wierszy dla kolumny {i}: ",df_order_payments[i].isna().sum())

Liczba pustych wierszy dla kolumny order_id:  0
Liczba pustych wierszy dla kolumny payment_sequential:  0
Liczba pustych wierszy dla kolumny payment_type:  0
Liczba pustych wierszy dla kolumny payment_installments:  0
Liczba pustych wierszy dla kolumny payment_value:  19572


In [10]:
print(len(df_order_items))
df_order_items["order_id"].nunique()

112650


98666

In [ ]:
df_order_items.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
payment_value          0
dtype: int64

Trzeba sprawdzić ile jest takich zamówień po order_id które mają dla jednego wiersza uzupełnioną cene a dla drugiego nie mają</br>
Tutaj trzeba będzie też sprawdzić coś takiego że jak w tabeli order_payments są wiersze które w ramach jednego zamówienia mają część danych uzupełnionych a część nie (kilka wiersz w tabeli order_payments to trzeba tą brakującą ilośc podzielić przez liczbę pustych wierszy i wstawić do tabeli payments)

In [14]:
df_order_payments.head(1)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33


Nastepnie trzeba bedzie podzielic roznice miedzy items a payments przez liczbe pustych wierszy dla danego order_id i uzupełnić brakujące wiersze dla payments

In [12]:
df_order_payments[df_order_payments["order_id"] == "fa65dad1b0e818e3ccc5cb0e39231352"]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
4885,fa65dad1b0e818e3ccc5cb0e39231352,1,voucher,1,3.71
7141,fa65dad1b0e818e3ccc5cb0e39231352,22,voucher,1,4.03
7224,fa65dad1b0e818e3ccc5cb0e39231352,23,voucher,1,18.86
7875,fa65dad1b0e818e3ccc5cb0e39231352,24,voucher,1,0.42
10147,fa65dad1b0e818e3ccc5cb0e39231352,25,voucher,1,3.68
12720,fa65dad1b0e818e3ccc5cb0e39231352,26,voucher,1,28.27
16191,fa65dad1b0e818e3ccc5cb0e39231352,2,voucher,1,NaN
20907,fa65dad1b0e818e3ccc5cb0e39231352,3,voucher,1,NaN
23860,fa65dad1b0e818e3ccc5cb0e39231352,4,voucher,1,NaN
26151,fa65dad1b0e818e3ccc5cb0e39231352,5,voucher,1,NaN


In [ ]:
order_payment_value_nulls = df_order_payments[df_order_payments["payment_value"].isna()]["order_id"]
part_empty_payment_value = df_order_payments[df_order_payments["order_id"].isin(order_payment_value_nulls)]
current_payment_val = part_empty_payment_value["payment_value"][part_empty_payment_value["order_id"] == "fa65dad1b0e818e3ccc5cb0e39231352"].sum() 
part_empty_payment_value # part_empty_payment_value zawiera takie order_id dla których wszystkie wiersze payment_value są nie uzupełnione 
# LUB takie order_id dla których payment_value jest dla niektórych wierszy częściowo uzupełnione a częściowo nie 

,order_id,payment_sequential,payment_type,payment_installments,payment_value
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,NaN
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,NaN
5,298fcdf1f73eb413e4d26d01b25bc1cd,1,credit_card,2,NaN
6,771ee386b001f06208a7419e4fc1bbd7,1,credit_card,1,NaN
7,3d7239c394a212faae122962df514ac7,1,credit_card,3,NaN
...,...,...,...,...,...
103525,59d02be1c50fb6e6930cafd6ff6a31e5,8,voucher,1,14.20
103551,086cb9cd0cf7067d77fe09f31620a52b,4,voucher,1,40.48
103795,c27a26005da9b758c804674cc9aa2da1,4,voucher,1,100.00
103805,d8d126f05b944cb1a5d78542e4a34f9a,2,voucher,1,37.66


In [14]:
mapping = df_order_items.groupby("order_id")["payment_value"].first().copy()
mapping

order_id
00010242fe8c5a6d1ba2dd792cb16214     72.19
00018f77f2f0320c557190d7a144bdd3    259.83
000229ec398224ef6ca0657da4fc703e    216.87
00024acbcdf0a6daa1e931b038114c75     25.78
00042b26cf59d7ce69dfabb4e55b4fd9    218.04
                                     ...  
fffc94f6ce00a00581880bf54a75a037    343.40
fffcd46ef2263f404302a634eb57f7eb    386.53
fffce4705a9662cd70adb13d4a31832d    116.85
fffe18544ffabc95dfada21779c9644f     64.71
fffe41c64501cc87c801fd61db3f6244     55.79
Name: payment_value, Length: 98666, dtype: float64

In [ ]:
full_payments_value = df_order_items.groupby("order_id")["payment_value"].sum()
#full_payments_value["fedcd9f7ccdc8cba3a18defedd1a5547"]

np.float64(205.73999999999998)

In [ ]:
for order_id, group in part_empty_payment_value.groupby(by="order_id"):
    # Jeśli grupa jest jednoelementowa to całość kwoty będzie brana z order_items
    try:
        if len(group) == 1:
            part_empty_payment_value.loc[group.index,"payment_value"] = group["payment_value"].fillna(group["order_id"].map(mapping))
        
        else: 
            current_payment_sum = group["payment_value"].sum()
            full_value = full_payments_value[group["order_id"]][0]
            mask = group["payment_value"].isna()
            nan_counts = len(mask[mask])
            value_to_fulfill = (full_value - current_payment_sum)/nan_counts
            part_empty_payment_value.loc[group.index,"payment_value"] = group["payment_value"].fillna(value_to_fulfill)
        
    except KeyError:
        continue

C:\Users\PW\AppData\Local\Temp\ipykernel_8168\3155147829.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  full_value = full_payments_value[group["order_id"]][0]
C:\Users\PW\AppData\Local\Temp\ipykernel_8168\3155147829.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  full_value = full_payments_value[group["order_id"]][0]
C:\Users\PW\AppData\Local\Temp\ipykernel_8168\3155147829.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]

In [ ]:
mask = part_empty_payment_value["payment_value"].isna()
part_empty_payment_value.loc[mask]["order_id"].value_counts()

order_id
0b398dbb3e7c81005f3c69b01488c4c9    3
cdf36cd4632fe33085adec5cf3d6b306    2
6ec704c9a120baaa6b3e162255ec684b    2
c037dd9db2ac27aa44381219612e91ba    1
bd7585a77f0aa76b3775bb68fb293356    1
                                   ..
d8a40a1358d3b2dca231abdaa566b054    1
15ec3d48383c58c0dc96cd66f8884630    1
115b01b1f77f0e02812425b6f5cd8f2f    1
85a22eb264ad44ff56c4955a66876fed    1
35de4050331c6c644cddc86f4f2d0d64    1
Name: count, Length: 152, dtype: int64

In [ ]:
part_empty_payment_value["payment_value"].isna().sum()

np.int64(156)

Po uzupełnieniu brakujących danych zostało 156 wierszy które nie mają uzupełnionych payment_value, z czego jest 152 unikalnych order_id (czyli takich pustych wierszy które nie mają swojego odzwierciedlenia w order_items - Te wiersze będzie można usunąć)

4. Usunięcie dla niektórych wartości z kolumny order_estimated_delivery_date dla tabeli 'olist_orders_dataset' i następnie uzupełnienie poprzez sumę order_purchase_timestamp + 30 dni (przyjmujemy takie założenie że gdzie jest nieuzupełnione tam będzie dostarczone za miesiąc) / LUB na podstawie mediany czasu dostarczenia zamówień dla tego seller_city i customer_city 

In [66]:
df_orders = pd.read_sql('SELECT * FROM dbo.orders', engine)
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17


In [67]:
df_orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date     339
dtype: int64

In [86]:
mask = df_orders["order_estimated_delivery_date"].isna() & df_orders["order_approved_at"].notna()
orders = df_orders.loc[mask,"order_id"].to_list()
orders[:5]

['00571ded73b3c061925584feab0db425',
 '01855f880aae9a984c7c33b26fcf2e02',
 '01a75ac3dd000b7628a684be97c29234',
 '02aec12849744e266394182861265286',
 '0341b60065c49221a27293793f11b117']

I teraz na podstawie tej listy zamówień trzeba obliczyć jaki jest średni czas dostarczenia zamówienia od danego seller_city do customer_city na podstawie zamówień archiwalnych </br>
W tym celu potrzebuje tabel Customers, Sellers, geolocation</br>
Sellers ma brakujące dane w city i state, dlatego trzeba się cofnąć i po I transoformacji zrobić transformacje dla tabeli Sellers (i wprowadzić brakujące dane)

In [154]:
df_customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [159]:
df_sellers = df_sellers.dropna()

In [161]:
df_sellers.isna().sum()

seller_zip_code_prefix    0
seller_id                 0
seller_city               0
seller_state              0
dtype: int64

In [156]:
df_geolocation.isna().sum()

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

In [162]:
orders[0]

'00571ded73b3c061925584feab0db425'

In [166]:
len(df_orders)

99441

In [ ]:
result = df_customers.merge(df_sellers, how = "left", on = "customer_id")
result

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP
...,...,...,...,...,...,...,...,...,...,...,...,...
99436,fffc94f6ce00a00581880bf54a75a037,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA
99437,fffcd46ef2263f404302a634eb57f7eb,84c5d4fbaf120aae381fad077416eaa0,delivered,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01,0da9fe112eae0c74d3ba1fe16de0988b,81690,curitiba,PR
99438,fffce4705a9662cd70adb13d4a31832d,29309aa813182aaddc9b259e31b870e6,delivered,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10,cd79b407828f02fdbba457111c38e4c4,4039,sao paulo,SP
99439,fffe18544ffabc95dfada21779c9644f,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25,eb803377c9315b564bdedad672039306,13289,vinhedo,SP
